# Produce the tables for the README.md of PMQD

This notebook requires `numpy`, `pandas` and `tabulate`. Uncomment below to install the requirements.

In [1]:
#!pip install numpy pandas tabulate

----------

In [2]:
import numpy as np
import pandas as pd

In [3]:
metadata = pd.read_csv("gs://pmqd/pmqd.csv", index_col="id")
metadata.sample(n=5)

,genre,artist,title,degradation_type,degradation_intensity,rating,sample_start,sample_filename
id,,,,,,,,
833,Reggae,Martin Landh,Bona Fide,noise,36.981741,3.0,86,d18a0226ff864e4f95e59193d0afaa0c.wav
631,Pop,Traditional,Hark! The Herald Angels Sing Holiday Edition (...,lowpass,14.966395,5.0,115,722507c5212243e1b8e40c658034b46d.wav
457,Acoustic,Anders Ekengren,Soft Nylon Guitar 2,limiter,24.392448,4.0,14,c9e88c60199243919e30de6ad5d75eb9.wav
588,Acoustic,Gavin Luke,A Magical Evening Begins,lowpass,90.542232,4.0,0,3c7064c701b948ffb7c37bbead63d75c.wav
171,Latin,Magnus Ringblom,Glamorous Housewives 6,original,0.000000,4.0,115,3ab9e6d35e804657befcd7c5722b04ce.wav


In [4]:
counts_by_genre = metadata.groupby("genre")["title"].count().reset_index()
counts_by_genre = counts_by_genre.rename(columns={"genre": "Genre", "title": "Count"})
counts_by_genre = counts_by_genre.append(
    {"Count": f"**{len(metadata)}**", "Genre": "**All**"}, ignore_index=True)
print(counts_by_genre.to_markdown(index=False))

| Genre               | Count   |
|:--------------------|:--------|
| Acoustic            | 75      |
| Blues               | 75      |
| Classical           | 75      |
| Country             | 75      |
| Electronica & Dance | 75      |
| Funk                | 75      |
| Hip Hop             | 75      |
| Jazz                | 75      |
| Latin               | 75      |
| Pop                 | 75      |
| Reggae              | 75      |
| Rnb & Soul          | 75      |
| Rock                | 75      |
| **All**             | **975** |


In [5]:
n = 5
metadata["degradation_intensity_range"] = pd.cut(
    metadata["degradation_intensity"],
    bins=np.linspace(0, 100, n + 1),
    right=False,
)

degradation_keys = ["degradation_type", "degradation_intensity_range"]
rating_by_degradation = metadata.groupby(degradation_keys)["rating"].mean().reset_index()
rating_by_degradation = rating_by_degradation.pivot(
    index="degradation_intensity_range", columns="degradation_type")
rating_by_degradation.columns = rating_by_degradation.columns.droplevel()
rating_by_degradation.index.name = "Degradation intensity"
rating_by_degradation.columns = rating_by_degradation.columns.str.capitalize()

# Replace NaN with None for better markdown table formatting
rating_by_degradation = rating_by_degradation.where(pd.notnull(rating_by_degradation), None)

print("Rating by degradation type and intensity\n")
print(rating_by_degradation.to_markdown(floatfmt=".2f", missingval="-"))

Rating by degradation type and intensity

| Degradation intensity   |   Distortion |   Limiter |   Lowpass |   Noise |   Original |
|:------------------------|-------------:|----------:|----------:|--------:|-----------:|
| [0.0, 20.0)             |         3.05 |      4.04 |      3.97 |    3.47 |       4.02 |
| [20.0, 40.0)            |         2.69 |      3.72 |      4.00 |    3.00 |       -    |
| [40.0, 60.0)            |         2.39 |      3.86 |      3.82 |    2.37 |       -    |
| [60.0, 80.0)            |         2.17 |      3.90 |      3.55 |    1.78 |       -    |
| [80.0, 100.0)           |         1.59 |      3.74 |      3.31 |    1.37 |       -    |
